In [164]:
import polars as pl
import altair as alt

alt.data_transformers.disable_max_rows()
alt.theme.enable('ggplot2')

pl.Config.set_fmt_str_lengths(100)

jco_colors = [
    "#0073C2",  # blue
    "#EFC000",  # yellow
    "#868686",  # gray
    "#CD534C",  # red
    "#7AA6DC",  # light blue
    "#003C67",  # dark blue
    "#8F7700",  # dark yellow
    "#3B3B3B",  # dark gray
    "#A73030",  # dark red
    "#4A6990"   # slate blue
]


file = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2025/2025-10-14/food_security.csv'

df = pl.read_csv(file, null_values=['NA'])

display(df.shape)

df.head(4)

(171232, 10)

Year_Start,Year_End,Area,Item,Unit,Value,CI_Lower,CI_Upper,Flag,Note
i64,i64,str,str,str,f64,str,str,str,str
2005,2007,"""Afghanistan""","""Average dietary energy supply adequacy (percent) (3-year average)""","""%""",97.0,null,null,"""Estimated value""",null
2006,2008,"""Afghanistan""","""Average dietary energy supply adequacy (percent) (3-year average)""","""%""",99.0,null,null,"""Estimated value""",null
2007,2009,"""Afghanistan""","""Average dietary energy supply adequacy (percent) (3-year average)""","""%""",102.0,null,null,"""Estimated value""",null
2008,2010,"""Afghanistan""","""Average dietary energy supply adequacy (percent) (3-year average)""","""%""",104.0,null,null,"""Estimated value""",null


In [169]:
assessment = 'Prevalence of severe food insecurity in the total population (percent) (annual value)'

chart = alt.Chart(
    df.filter(
        pl.col('Item') == assessment,
        pl.col('Area').str.contains('economies'),
    ).with_columns(
        pl.col('Area').str.split(' ').list.first(),
        pl.col('Value') / 100.0,
    ),
    title=alt.Title(
        'Food insecurity is getting worse in lower income economies across the world, not better.',
        subtitle=[
            'Assessment: "Prevalence of severe food insecurity in the total population (%)"',
            'Source: The Food and Agriculture Organization of the United Nations (FAO)',
        ],
    ),
    width=600,
    height=550,
    background='#fdfbf2',
).mark_line(
    size=3.5,
).encode(
    x=alt.X('Year_Start', title='', axis=alt.Axis(format='d')),
    y=alt.Y('Value', title='percentage of population', axis=alt.Axis(format='%')),
    color=alt.Color(
        'Area',
        title='',
        scale=alt.Scale(domain=['Low-income', 'Lower-middle-income', 'Upper-middle-income', 'High-income'], range=jco_colors),
        legend=alt.Legend(symbolType='stroke', symbolStrokeWidth=4, labelFontSize=12),
    ),
)

display(chart)

alt.Chart(...)